In [43]:
from collections import defaultdict
from collections import deque, OrderedDict
from itertools import islice
class BaseSampler(object):
    def __init__(self,
                 max_path_length,
                 environment=None,
                 policy=None,
                 pool=None,
                 store_last_n_paths=10):
        self._max_path_length = max_path_length
        self._store_last_n_paths = store_last_n_paths
        self._last_n_paths = deque(maxlen=store_last_n_paths)

        self.environment = environment
        self.policy = policy
        self.pool = pool

    def initialize(self, environment, policy, pool):
        self.environment = environment
        self.policy = policy
        self.pool = pool

    def reset(self):
        pass

    def set_policy(self, policy):
        self.policy = policy

    def clear_last_n_paths(self):
        self._last_n_paths.clear()

    def get_last_n_paths(self, n=None):
        if n is None:
            n = self._store_last_n_paths

        last_n_paths = tuple(islice(self._last_n_paths, None, n))

        return last_n_paths

    def sample(self):
        raise NotImplementedError

    def terminate(self):
        self.environment.close()

    def get_diagnostics(self):
        diagnostics = OrderedDict({'pool-size': self.pool.size})
        return diagnostics

    def __getstate__(self):
        state = {
            key: value for key, value in self.__dict__.items()
            if key not in (
                'environment',
                'policy',
                'pool',
                '_last_n_paths',
                '_current_observation',
                '_current_path',
                '_is_first_step',
            )
        }

        return state

    def __setstate__(self, state):
        self.__dict__.update(state)

        self.environment = None
        self.policy = None
        self.pool = None
        # TODO(hartikainen): Maybe try restoring these from the pool?
        self._last_n_paths = deque(maxlen=self._store_last_n_paths)
import numpy as np
import tree
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import gzip
import pickle

import numpy as np
import tensorflow as tf
import tree
import abc
from gym import spaces
import numpy as np

In [44]:
a = [[1,2,3],[4,5,6]]
b=np.stack(a,axis=0)
print(b)

[[1 2 3]
 [4 5 6]]


In [45]:
import os
import time
import grid2op
import numpy as np
import re
from grid2op.Parameters import Parameters
from grid2op.Reward import BaseReward, RedispReward, L2RPNSandBoxScore
from multiprocessing import cpu_count,Pool
import grid2op
from grid2op.Reward import BaseReward, RedispReward, L2RPNSandBoxScore
import numpy as np
from grid2op.Parameters import Parameters


other_rewards = {}
other_rewards["tmp_score_codalab"] = L2RPNSandBoxScore
input_dir = '../input_data_local'
parameters = Parameters()
parameters.HARD_OVERFLOW_THRESHOLD = 3.0
parameters.MAX_SUB_CHANGED = 6
parameters.NB_TIMESTEP_OVERFLOW_ALLOWED = 2
parameters.MAX_LINE_STATUS_CHANGED = 100

env = grid2op.make(input_dir, param=parameters,
                   reward_class=RedispReward,
                   other_rewards=other_rewards)
env.seed(0)
env.set_id(0)
obs = env.reset()

In [46]:
action = env.action_space({})
env.step(action)
obs = env.get_obs()
obs_handle = np.concatenate(([obs.rho.max()],obs.line_status))
obs, reward, terminal, info = env.step(action)
obs_next = np.concatenate(([obs.rho.max()],obs.line_status))

In [47]:
def _process_sample(observation,
                    action,
                    reward,
                    terminal,
                    next_observation,
                    info):
    processed_observation = {
        'observations': observation,
        'actions': action,
        'rewards': np.atleast_1d(reward),
        'terminals': np.atleast_1d(terminal),
        'next_observations': next_observation,
        'infos': info,
    }

    return processed_observation

processed_sample = _process_sample(
    observation=obs_handle,
    action=[0],
    reward=reward,
    terminal=terminal,
    next_observation=obs_next,
    info=info,
)

In [48]:
_current_path = []
for i in range(4):
    action = env.action_space({})
    env.step(action)
    obs = env.get_obs()
    obs_handle = np.concatenate(([obs.rho.max()],[obs.rho.min()]))
    obs, reward, terminal, info = env.step(action)
    obs_next = np.concatenate(([obs.rho.max()],[obs.rho.min()]))
    processed_sample = _process_sample(
        observation=obs_handle,
        action=[0],
        reward=reward,
        terminal=terminal,
        next_observation=obs_next,
        info=info,
    )
    _current_path.append(processed_sample)

print(_current_path)




[{'observations': array([0.62514377, 0.02076853], dtype=float32), 'actions': [0], 'rewards': array([935.14124], dtype=float32), 'terminals': array([False]), 'next_observations': array([0.6261801 , 0.01758549], dtype=float32), 'infos': {'disc_lines': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': False, 'is_illegal_reco': False, 'reason_alarm_illegal': None, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [], 'rewards': {'tmp_score_codalab': 370.40283}}}, {'observations': array([0.62673396, 0.01941825], dtype=float32), 'actions': [0], 'rewards': array([933.26385], dtype=float32), 'terminals': array([False]), 'next_observations': array([0.62328833, 0

In [49]:
import tree
# 这里的path指的是数据
last_path = tree.map_structure(
    lambda *x: np.stack(x, axis=0), *_current_path)
print(last_path)

{'observations': array([[0.62514377, 0.02076853],
       [0.62673396, 0.01941825],
       [0.623004  , 0.01229346],
       [0.62107897, 0.01495934]], dtype=float32), 'actions': [array([0, 0, 0, 0])], 'rewards': array([[935.14124],
       [933.26385],
       [936.9162 ],
       [936.8206 ]], dtype=float32), 'terminals': array([[False],
       [False],
       [False],
       [False]]), 'next_observations': array([[0.6261801 , 0.01758549],
       [0.62328833, 0.01678412],
       [0.62348497, 0.01932848],
       [0.6189333 , 0.01083042]], dtype=float32), 'infos': {'disc_lines': array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
      

In [50]:
path = last_path.copy()


@dataclass
class Field:
    name: str
    dtype: Union[str, np.dtype, tf.DType]
    shape: Union[tuple, tf.TensorShape]
    initializer: Callable = np.zeros
    default_value: Number = 0.0

INDEX_FIELDS = {
    'episode_index_forwards': Field(
        name='episode_index_forwards',
        dtype='uint64',
        shape=(1, ),
        default_value=0,
    ),
    'episode_index_backwards': Field(
        name='episode_index_backwards',
        dtype='uint64',
        shape=(1, ),
        default_value=0,
    ),
}

fields_ = {}
def _initialize_field(self, field):
    field_shape = (self._max_size, *field.shape)
    field_values = field.initializer(
        field_shape, dtype=field.dtype)

    return field_values
def add_samples(samples):
    num_samples = tree.flatten(samples)[0].shape[0]

    fields = {**fields_, **INDEX_FIELDS}
    data = tree.map_structure(_initialize_field, fields)

    assert (('episode_index_forwards' in samples.keys())
            is ('episode_index_backwards' in samples.keys()))
    _pointer = 10
    _max_size = 200
    index = np.arange(
        _pointer, _pointer + num_samples) % _max_size

    def add_sample(path, data, new_values, field):
        assert new_values.shape[0] == num_samples, (
            new_values.shape, num_samples)
        data[index] = new_values

    tree.map_structure_with_path(
        add_sample, self.data, samples, self.fields)

    # self._advance(num_samples)

def add_path(path):
    path = path.copy()
    path_length = tree.flatten(path)[0].shape[0]
    path.update({
        'episode_index_forwards': np.arange(
            path_length,
            dtype='uint64'
        )[..., np.newaxis],
        'episode_index_backwards': np.arange(
            path_length,
            dtype='uint64'
        )[::-1, np.newaxis],
    })

    print(path)
    # add_samples(path)



add_path(path)



{'observations': array([[0.62514377, 0.02076853],
       [0.62673396, 0.01941825],
       [0.623004  , 0.01229346],
       [0.62107897, 0.01495934]], dtype=float32), 'actions': [array([0, 0, 0, 0])], 'rewards': array([[935.14124],
       [933.26385],
       [936.9162 ],
       [936.8206 ]], dtype=float32), 'terminals': array([[False],
       [False],
       [False],
       [False]]), 'next_observations': array([[0.6261801 , 0.01758549],
       [0.62328833, 0.01678412],
       [0.62348497, 0.01932848],
       [0.6189333 , 0.01083042]], dtype=float32), 'infos': {'disc_lines': array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
      

In [51]:
import gym
env = gym.make('CartPole-v0')
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)
print(env.action_space)

def field_from_gym_space(name, space):
    if isinstance(space, spaces.Box):
        if isinstance(name, (list, tuple)):
            name = '/'.join(name)
        print("name:{}".format(name))
        return Field(name=name, dtype=space.dtype, shape=space.shape)
    elif isinstance(space, spaces.Dict):
        return tree.map_structure_with_path(
            field_from_gym_space, space.spaces)
    else:
        raise NotImplementedError(space)

Box(4,)
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
Discrete(2)


In [67]:
obs = field_from_gym_space('observations', env.observation_space)
print(env.action_space.dtype)
print(env.action_space.shape)
Field(
    name='actions',
    dtype=env.action_space.dtype,
    shape=env.action_space.shape)

print(obs_handle.dtype)
print(obs_handle.shape)
action_space = np.array([0])
print(action_space.dtype)
print(action_space.shape)

name:observations
int64
()
float32
(2,)
int32
(1,)
